# Using Customized Data To Run PywrDRB

PywrDRB provides multiple `inflow_type` and `diversion_type` options for users to select directly during the model-building process. However, sometimes users may want to use their own customized data to run the model.

In this tutorial, we will walk you through:  
1) The path structure adopted in PywrDRB.  
2) How to use your own flow and diversion data to run the model.


## Path Structure in PywrDRB

In PywrDRB, we use a global instance of the [PathNavigator](https://github.com/philip928lin/PathNavigator) object to manage the paths associated with different datasets used in PywrDRB.

To get the customizable path configuration, you can do:


In [3]:
import pywrdrb
from pprint import pprint

pn_config = pywrdrb.get_pn_config()
pprint(pn_config)

{'flows/nhmv10': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\nhmv10',
 'flows/nhmv10_withObsScaled': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\nhmv10_withObsScaled',
 'flows/nwmv21': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\nwmv21',
 'flows/nwmv21_withObsScaled': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\nwmv21_withObsScaled',
 'flows/pub_nhmv10_BC_withObsScaled': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\pub_nhmv10_BC_withObsScaled',
 'flows/wrf1960s_calib_nlcd2016': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\wrf1960s_calib_nlcd2016',
 'flows/wrf2050s_calib_nlcd2016': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\wrf2050s_calib_nlcd2016',
 'flows/wrfaorc_calib_nlcd2016': 'C:\\Users\\CL\\Documents\\GitHub\\Pywr-DRB\\src\\pywrdrb\\data\\flows\\wrfaorc_calib_nlcd2016',
 'flows/wrfaorc_wi

From here, you can see that we provide five inflow types: `nhmv10`, `nhmv10_withObsScaled`, `nwmv21`, `nwmv21_withObsScaled`, `wrf1960s_calib_nlcd2016`, `wrf2050s_calib_nlcd2016`, and `wrfaorc_calib_nlcd2016`, along with the corresponding five diversion types: `nhmv10`, `nhmv10_withObsScaled`, `nwmv21`, `nwmv21_withObsScaled`, `wrf1960s_calib_nlcd2016`, `wrf2050s_calib_nlcd2016`, and `wrfaorc_calib_nlcd2016`. 

The directories of the corresponding folders are stored in a dictionary, where the keys have the prefix `flows/` and `diversions/` to distinguish between flow data and diversion data.

So, if you want to use your own data to run the simulation, you will need to add your folder directory to `pn_config` and load it into `pywrdrb` before building and running the model. 

Let me show you how. Assuming you want to use your own flow datasets (we will discuss the required datasets soon) stored in an external folder `C:/my_data`, you need to add `{"flows/my_data": "C:/my_data"}` to `pn_config`. Then, you can use `my_data` as the flow type when using the `modelbuilder`.


In [ ]:
# Before adding the custom config if we try to use the custom inflow type, it will raise an error

r"""
mb = pywrdrb.ModelBuilder(
    inflow_type='my_data', 
    diversion_type='nhmv10',
    start_date="1983-10-01",
    end_date="1985-12-31"
    )
"""
# Make a model (you are expected to see error here if you uncomment the line below)
# mb.make_model()

In [ ]:
# For demonstration purpose, let me assume my_data folder has the same directory of 
# nhmv10 flow type (you will use your actual directory to the folder)
pn_config = pywrdrb.get_pn_config()
pn_config["flows/my_data"] = pn_config["flows/nhmv10"]

pywrdrb.load_pn_config(pn_config)

# Now we can use the custom inflow type
mb = pywrdrb.ModelBuilder(
    inflow_type='my_data', 
    diversion_type='nhmv10',
    start_date="1983-10-01",
    end_date="1985-12-31"
    )

# Make a model (you are expected to see error here)
mb.make_model()
print("You secussessfully created a model with custom inflow type")

## Datasets you need to have in your customize "my_folder"



In [20]:
import pandas as pd
print("For flow type folder, you need to have the following files: \n")
pn = pywrdrb.get_pn_object()
files = pn.data.flows.nhmv10.list()
print(f"File needed: {files}\n")  
for file in files:
    df = pd.read_csv(pn.data.flows.nhmv10.get(file))
    print(f"File: {file}")
    print(df.iloc[:5, :5]) # print first 5 rows and 5 columns
    print("\n")
#df.head()

For flow type folder, you need to have the following files: 

File needed: ['catchment_inflow_mgd.csv', 'gage_flow_mgd.csv', 'predicted_inflows_mgd.csv']

File: catchment_inflow_mgd.csv
     datetime  cannonsville    pepacton   neversink  wallenpaupack
0  1980-10-01    130.379373   89.325193   60.718368      33.438627
1  1980-10-02    320.876434  234.425929   58.710650     155.951484
2  1980-10-03    384.188487  261.300230   71.301161     196.969214
3  1980-10-04    345.220715  265.843778  115.661384     229.965044
4  1980-10-05    322.869093  257.040294   71.100223     208.087549


File: gage_flow_mgd.csv
     datetime  cannonsville    pepacton   neversink  wallenpaupack
0  1980-10-01    130.379373   89.325193   60.718368      33.438627
1  1980-10-02    320.876434  234.425929   58.710650     155.951484
2  1980-10-03    384.188487  261.300230   71.301161     196.969214
3  1980-10-04    345.220715  265.843778  115.661384     229.965044
4  1980-10-05    322.869093  257.040294   71.100223

In [21]:
import pandas as pd
print("For diversions type folder, you need to have the following files: \n")
pn = pywrdrb.get_pn_object()
files = pn.data.diversions.nhmv10.list()
print(f"File needed: {files}\n")  
for file in files:
    df = pd.read_csv(pn.data.diversions.nhmv10.get(file))
    print(f"File: {file}")
    print(df.iloc[:5, :5]) # print first 5 rows and 5 columns
    print("\n")
#df.head()

For diversions type folder, you need to have the following files: 

File needed: ['diversion_nj_extrapolated_mgd.csv', 'diversion_nyc_extrapolated_mgd.csv', 'predicted_diversions_mgd.csv']

File: diversion_nj_extrapolated_mgd.csv
     datetime  D_R_Canal
0  1945-01-01  85.387245
1  1945-01-02  80.717630
2  1945-01-03  60.037906
3  1945-01-04  48.030325
4  1945-01-05  56.035379


File: diversion_nyc_extrapolated_mgd.csv
     datetime  cannonsville  pepacton  neversink   aggregate
0  1945-01-01           NaN       NaN        NaN  501.333527
1  1945-01-02           NaN       NaN        NaN  501.333527
2  1945-01-03           NaN       NaN        NaN  501.333527
3  1945-01-04           NaN       NaN        NaN  805.273987
4  1945-01-05           NaN       NaN        NaN  837.798981


File: predicted_diversions_mgd.csv
    datetime  demand_nj_lag1_regression_disagg  \
0  10/1/1983                         91.425844   
1  10/2/1983                         88.815772   
2  10/3/1983            

## More About the Global PathNavigator Object Used in PywrDRB

We can get the global `PathNavigator` object used in PywrDRB by running: `pn = pywrdrb.get_pn_object()`

This `pn` object contains all the directory and path information, allowing you to locate specific files used in PywrDRB within the file explorer.

More `pn` operations can be found [here](https://github.com/philip928lin/PathNavigator). However, users should ONLY use pn to explore file and folder locations. It is not designed for modifications unless you fully understand what you are doing.

In [ ]:
pn = pywrdrb.get_pn_object()
print(f"The root directory of the pywrdrb: {pn.get()}")

The root directory of the pywrdrb: pn.get()


In [24]:
pn.tree()

pywrdrb
├── data
│   ├── catchment_withdrawals
│   │   └── sw_avg_wateruse_pywrdrb_catchments_mgd.csv
│   ├── diversions
│   │   ├── nhmv10
│   │   │   ├── diversion_nj_extrapolated_mgd.csv
│   │   │   ├── diversion_nyc_extrapolated_mgd.csv
│   │   │   └── predicted_diversions_mgd.csv
│   │   ├── nhmv10_withObsScaled
│   │   │   ├── diversion_nj_extrapolated_mgd.csv
│   │   │   ├── diversion_nyc_extrapolated_mgd.csv
│   │   │   └── predicted_diversions_mgd.csv
│   │   ├── nwmv21
│   │   │   ├── diversion_nj_extrapolated_mgd.csv
│   │   │   ├── diversion_nyc_extrapolated_mgd.csv
│   │   │   └── predicted_diversions_mgd.csv
│   │   ├── nwmv21_withObsScaled
│   │   │   ├── diversion_nj_extrapolated_mgd.csv
│   │   │   ├── diversion_nyc_extrapolated_mgd.csv
│   │   │   └── predicted_diversions_mgd.csv
│   │   ├── wrf1960s_calib_nlcd2016
│   │   │   ├── diversion_nj_extrapolated_mgd.csv
│   │   │   ├── diversion_nyc_extrapolated_mgd.csv
│   │   │   └── predicted_diversions_mgd.csv
│   │   ├